In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from PIL import Image
import pandas as pd
import requests
import numpy as np

In [6]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(y_test.shape)) 

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [8]:
# Apply augmentation to the whole training dataset
# Define the augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),         # Random rotation (20 degrees)
    tf.keras.layers.RandomTranslation(0.1, 0.1), # Random horizontal and vertical shift
    tf.keras.layers.RandomZoom(0.2),             # Random zoom
    tf.keras.layers.RandomFlip('horizontal')     # Random horizontal flip
])

# Create variables to hold the X and y training data
X_train_aug = []
y_train_aug = []

# Loop through all the images.
for i in range(len(X_train)):
    # Select the image
    img = X_train[i]
    # Select the label from the training data
    label = y_train[i]

    # Add a channel dimension for grayscale images
    img = np.expand_dims(img, axis=-1)  # Add channel dimension

    # Ensure that the input data has the correct shape
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Add 5 images for every original image
    for j in range(5):
        # Append a new image to the X list
        X_train_aug.append(data_augmentation(img, training=True)[0].numpy())
        # Append the label for the original image to the y list
        y_train_aug.append(label)

# Print the length of each list
print(len(X_train_aug))
print(len(y_train_aug))

300000
300000


In [9]:
# Reshape test data for the model
X_test_np = []
for img in X_test:
    # Add a channel dimension for grayscale images
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    # Append the image to the list
    X_test_np.append(img)

# Convert to numpy array
X_test_np = np.array(X_test_np)

# Check the shape of the first image
X_test_np[0].shape

(28, 28, 1)

In [ ]:
# One hot encode the y data
y_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(np.array(y_train_aug).reshape(-1, 1))
y_train_aug_enc = y_encoder.transform(np.array(y_train_aug).reshape(-1, 1))
y_test_enc = y_encoder.transform(np.array(y_test).reshape(-1, 1))

# Convert values to numpy arrays
X_train_aug_np = np.array(X_train_aug)
X_test_np = np.array(X_test_np)
y_train_aug_np = np.array(y_train_aug_enc)
y_test_np = np.array(y_test_enc)


# Split the training dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_aug_np, y_train_aug_np, test_size=0.2, random_state=42)

# Print the total number of one_hot_encoded columns
np.array(y_train).shape

(240000, 10)

In [15]:
# Define a CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='sigmoid')  # 10 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs
)

Epoch 1/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 44s 6ms/step - accuracy: 0.7199 - loss: 0.9055 - val_accuracy: 0.9155 - val_loss: 0.2688
Epoch 2/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.9243 - loss: 0.2433 - val_accuracy: 0.9348 - val_loss: 0.2149
Epoch 3/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - accuracy: 0.9404 - loss: 0.1920 - val_accuracy: 0.9370 - val_loss: 0.2073
Epoch 4/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 44s 6ms/step - accuracy: 0.9480 - loss: 0.1662 - val_accuracy: 0.9411 - val_loss: 0.1982
Epoch 5/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 41s 6ms/step - accuracy: 0.9526 - loss: 0.1532 - val_accuracy: 0.9425 - val_loss: 0.1913
Epoch 6/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.9554 - loss: 0.1415 - val_accuracy: 0.9402 - val_loss: 0.2054
Epoch 7/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 42s 6ms/step - accuracy: 0.9582 - loss: 0.1344 - val_accuracy: 0.9432 - val_loss: 0.1930
Epoch 8/10
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - accuracy: 0.9598 - loss: 0

In [16]:
# Evaluate the model using the testing data
model.evaluate(X_test_np, y_test_np)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9137 - loss: 0.4434


[0.38629186153411865, 0.9228000044822693]